In [ ]:
%matplotlib inline

import glob
import matplotlib.pylab as plt
import numpy as np
import os
import struct
import warnings

from __future__ import print_function
from matplotlib.mlab import detrend
from obspy import *
from obspy.core import read, UTCDateTime
from obspy.clients.fdsn import Client
from obspy.signal.cross_correlation import xcorr_pick_correction
from obspy.signal.filter import lowpass
from obspy.signal.invsim import cosine_taper
from scipy import interpolate, signal
from time import *

from lib_seis import smooth
from lib_seis import wf_fft

warnings.filterwarnings('ignore')                  # do not show warnings

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['lines.linewidth'] = 0.5

# Adaptation of CAN_noise.m by Carl Tape
# Coding by: Yongki Aiman, Tomy Gunawan, Angel Ling
# Analyzing the noise time series at CAN
# custom functions: wf_fft, smooth

In [ ]:
# get the waveform
client = Client("IRIS")
t = UTCDateTime("2004-12-26T00:58:53.0")
starttime = t-(9*24*3600)    # 9 days before the Sumatra earthquake
endtime = t                  # origin time of Sumatra earthquake

st = client.get_waveforms("G", "CAN", "*", "LHZ", starttime, endtime, attach_response=True)
print(st)

In [ ]:
# plot seismogram
tr = st[0]
times = st[0].times()
trace_data = tr.data

# FIX: do not hardcode station/channel
title = 'CAN (LHZ) − starting '+ str(st[0].stats['starttime'])
plt.plot(times,trace_data,color='b')
plt.title(title)
plt.xlabel('seconds')
plt.ylabel('counts')
plt.show();

In [ ]:
# extract a subset time series that has NO visible earthquakes
t1 = starttime + 5.0E5
t2 = starttime + 7.0E5
st1 = st.copy()
print(st1)
st1.trim(t1, t2)
print(st1)

In [ ]:
# FIX: do not hardcode station/channel
title = 'CAN (LHZ) − starting '+ str(st1[0].stats['starttime'])
plt.plot(st1[0].times(),st1[0].data,color='b')
plt.title(title)
plt.xlabel('seconds')
plt.ylabel('counts')
plt.show();

In [ ]:
# Spectral analysis and deconvolution starts here

